In [1]:
# incomplete NTT
from Poly import *

logn = 5
n = 1<<logn 

p = 16760833
R_p = quotient(ZZ,p*ZZ)
R_p_poly = quotient(PolynomialRing(R_p,x),x^n+1)
omega = R_p.zeta(n>>2) 
#print([omega^i for i in range((n>>2)+1)])

omegas = [omega^i for i in range((n>>2))]
print(n)
print((omegas))

32
[1, 9791491, 7137274, 9529539, 16760832, 6969342, 9623559, 7231294]


In [2]:
def extend(powers):
    new_powers = []
    d = len(powers)
    for i in range(d):
        new_powers.append(powers[i])
        new_powers.append(powers[i]+2*d)
    return new_powers

powers = [1]
list_of_powers = [powers]

for i in range(logn-3):
    powers = extend(powers)
    list_of_powers.append(powers)

total_power = list_of_powers[-1]
print((total_power))

[1, 5, 3, 7]


In [3]:
def StepDown(layer,f):
    """
    from {layer} to {layer+1}
    """
    # Twiddle
    step_size = 1 << (logn - 3 - layer)
    
    ptr = n>>(layer+1)
    for j in range(1<<layer):
        for i in range((n>>(layer+1))):
            f[ptr+i] = f[ptr+i] * omegas[(n>>(layer+4))*total_power[j*(step_size)]]
            
        ptr += (n>>layer)


    #CT Butterfly
    f_temp = f.copy()
    d = (n>>(layer+1)) #distance
    ptr = 0
    for j in range(2^layer):
        for i in range((n>>(layer+1))):
            f[ptr+  i] = f_temp[ptr+i] + f_temp[ptr+d+i]
            f[ptr+d+i] = f_temp[ptr+i] - f_temp[ptr+d+i]
            
        ptr += (n>>layer)


def StepUp(layer,f):
    """
    from {layer+1} to {layer}
    """
    
    #CT Butterfly
    f_temp = f.copy()
    d = (n>>(layer+1)) #distance
    ptr = 0
    for j in range(1 << layer):
        for i in range((n>>(layer+1))):
            f[ptr+  i] = f_temp[ptr+i] + f_temp[ptr+d+i]
            f[ptr+d+i] = f_temp[ptr+i] - f_temp[ptr+d+i]
            
        ptr += (n>>layer)


    
    # Twiddle
    step_size = 1 << (logn - 3 - layer)
    
    ptr = n>>(layer+1)
    for j in range(1 << layer):
        for i in range((n>>(layer+1))):
            f[ptr+i] = f[ptr+i] * omegas[(n>>2)-((n>>(layer+4))*total_power[j*(step_size)])]
            
        ptr += (n>>layer)

    # Normalization
    for i in range(n):
        f[i] = f[i] / R_p(2)

In [4]:
# Perform TMVP
def TMVP_base(f_small_8, g_small_8,twiddle):
    """
    Implement the TMVP.
    """
    result = [0]*8
    toep = [item * twiddle for item in f_small_8][1:] + f_small_8 
    for i in range(8):
        result[i] = sum([g_small_8[j] * toep[i+7-j] for j in range(8)])
    return result

In [5]:

f = Polynomial_Generator(n)
g = Polynomial_Generator(n)

#f = [1, 3, 0, 0, 3, 0, 1, 2, 2, 0, 2, 2, 2, 5, 1, 0, 0, 0, 3, 1, 1, 2, 0, 1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 3, 4, 3, 2, 0, 2, 0, 1, 6, 0, 0, 0, 3, 0, 2, 2, 5, 1, 0, 2, 2, 0, 1, 4, 0, 0, 4, 0, 0, 2, 2, 1, 5, 0, 0, 3, 2, 1, 6, 1, 5, 0, 2, 6, 0, 2, 1, 0, 4, 3, 1, 3, 0, 0, 0, 2, 3, 4, 3, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 4, 6, 1, 3, 1, 4, 3, 2, 0, 0, 2, 6, 0, 2, 0, 2, 2, 0, 0, 0, 0, 5, 2, 1, 2, 0, 2, 0, 2, 4, 2, 1, 2, 2, 2, 6, 5, 1, 0, 2, 1, 1, 1, 7, 1, 1, 2, 0, 2, 2, 2, 1, 5, 1, 3, 2, 0, 2, 3, 5, 4, 2, 1, 0, 1, 3, 1, 3, 4, 2, 0, 0, 2, 1, 4, 2, 2, 3, 1, 1, 0, 1, 2, 2, 2, 3, 2, 0, 4, 1, 1, 1, 3, 0, 2, 5, 1, 2, 4, 0, 1, 1, 0, 0, 0, 1, 0, 3, 2, 2, 1, 7, 1, 2, 1, 1, 3, 6, 5, 1, 0, 0, 1, 0, 1, 0, 1, 3, 5, 0, 1, 5, 1, 1, 2, 0, 3, 0, 0, 0, 0, 0, 1, 3, 0, 0, 0, 2, 3, 6, 4, 6, 2, 1]
#g = [3, 3, 1, 2, 1, 2, 1, 0, 2, 0, 0, 6, 4, 0, 8, 0, 0, 0, 1, 1, 1, 5, 2, 0, 4, 0, 2, 1, 2, 1, 2, 2, 0, 2, 0, 3, 4, 0, 3, 7, 1, 0, 0, 0, 2, 1, 3, 6, 2, 6, 1, 0, 2, 5, 0, 0, 0, 5, 2, 1, 2, 0, 3, 1, 7, 0, 1, 6, 0, 0, 1, 7, 1, 2, 1, 0, 3, 2, 4, 3, 8, 2, 3, 1, 1, 0, 2, 1, 0, 1, 0, 0, 4, 5, 1, 5, 3, 3, 2, 0, 0, 2, 0, 1, 3, 0, 1, 4, 1, 1, 1, 3, 2, 4, 3, 1, 3, 2, 1, 3, 2, 2, 3, 2, 1, 1, 4, 0, 1, 0, 1, 0, 1, 3, 5, 3, 1, 1, 2, 1, 1, 2, 0, 1, 5, 3, 0, 2, 6, 0, 1, 3, 0, 4, 2, 1, 2, 0, 0, 1, 1, 5, 3, 7, 5, 0, 4, 0, 3, 2, 2, 2, 0, 2, 1, 1, 1, 1, 0, 1, 0, 3, 3, 1, 1, 3, 0, 1, 0, 0, 4, 0, 0, 1, 0, 0, 3, 0, 0, 4, 0, 2, 0, 1, 0, 0, 2, 2, 1, 0, 3, 1, 2, 0, 4, 0, 0, 2, 1, 3, 0, 1, 4, 2, 4, 0, 3, 1, 6, 3, 1, 3, 1, 0, 5, 2, 2, 3, 5, 1, 1, 2, 3, 0, 5, 6, 0, 1, 1, 2, 6, 1, 3, 2, 1, 3]

#f = [1, 0, 2, 4, 1, 0, 0, 3, 0, 2, 5, 3, 0, 1, 2, 5, 3, 0, 3, 2, 1, 0, 1, 2, 2, 1, 3, 0, 0, 1, 1, 0, 2, 4, 1, 0, 1, 2, 3, 1, 0, 2, 0, 1, 0, 5, 2, 3, 2, 3, 1, 2, 2, 2, 3, 0, 4, 2, 1, 1, 1, 1, 1, 5, 4, 0, 0, 3, 0, 6, 0, 7, 4, 7, 2, 1, 1, 3, 1, 3, 4, 2, 4, 2, 3, 2, 4, 6, 3, 6, 3, 1, 0, 0, 1, 0, 3, 4, 3, 0, 0, 2, 7, 2, 6, 5, 4, 1, 1, 0, 1, 1, 3, 0, 0, 0, 2, 0, 1, 5, 1, 1, 1, 2, 2, 0, 4, 1, 0, 1, 0, 2, 5, 2, 2, 3, 2, 1, 4, 2, 3, 4, 8, 3, 1, 0, 3, 0, 5, 3, 1, 1, 1, 6, 7, 0, 1, 2, 2, 5, 1, 6, 6, 0, 3, 0, 1, 4, 1, 0, 6, 0, 0, 5, 0, 2, 3, 0, 2, 0, 1, 3, 1, 0, 1, 1, 0, 7, 0, 1, 2, 2, 0, 3, 0, 1, 0, 0, 1, 3, 0, 1, 5, 4, 1, 0, 0, 4, 1, 4, 5, 3, 2, 3, 4, 3, 0, 0, 1, 1, 1, 0, 4, 2, 3, 4, 2, 0, 1, 2, 0, 3, 3, 2, 4, 2, 1, 2, 1, 2, 3, 0, 5, 5, 2, 2, 4, 3, 1, 1, 5, 1, 1, 0, 4, 3]
#g = [3, 0, 1, 0, 3, 3, 2, 2, 1, 2, 3, 0, 1, 1, 4, 4, 2, 3, 4, 2, 2, 1, 3, 3, 3, 1, 2, 0, 0, 0, 0, 1, 1, 3, 5, 2, 0, 6, 3, 2, 2, 6, 2, 5, 1, 3, 0, 1, 6, 1, 3, 1, 7, 0, 3, 6, 0, 1, 0, 2, 2, 0, 0, 1, 1, 0, 0, 2, 5, 4, 6, 6, 0, 2, 4, 1, 0, 0, 3, 3, 5, 1, 1, 3, 0, 0, 3, 5, 1, 1, 1, 6, 1, 0, 3, 2, 4, 1, 0, 0, 3, 1, 1, 5, 0, 2, 0, 0, 0, 0, 2, 1, 1, 0, 3, 4, 2, 0, 4, 3, 1, 5, 1, 5, 5, 2, 8, 1, 4, 6, 3, 2, 0, 1, 1, 2, 2, 3, 0, 1, 1, 0, 2, 0, 0, 2, 0, 5, 3, 2, 1, 0, 3, 1, 3, 1, 1, 0, 2, 6, 0, 0, 0, 2, 0, 5, 3, 2, 1, 8, 6, 4, 4, 2, 3, 0, 4, 2, 1, 4, 0, 2, 0, 0, 1, 3, 0, 1, 1, 1, 5, 4, 6, 0, 1, 0, 2, 8, 0, 4, 1, 1, 4, 1, 0, 0, 2, 2, 0, 1, 2, 4, 0, 0, 0, 1, 0, 3, 1, 3, 2, 4, 2, 6, 2, 1, 2, 0, 2, 0, 0, 4, 0, 3, 0, 6, 2, 4, 1, 0, 7, 0, 0, 6, 4, 3, 1, 0, 4, 2, 2, 0, 1, 1, 0, 0]

f = [R_p(i) for i in f]
g = [R_p(i) for i in g]

f_poly = R_p_poly(f)
g_poly = R_p_poly(g)

print(f,g)

[1, 6, 4, 3, 4, 0, 2, 0, 4, 2, 3, 0, 1, 0, 0, 8, 1, 3, 0, 3, 1, 4, 1, 2, 3, 2, 0, 0, 1, 2, 0, 3] [0, 0, 4, 0, 0, 2, 1, 3, 2, 6, 4, 1, 1, 7, 3, 2, 0, 4, 0, 1, 0, 0, 6, 0, 2, 0, 1, 3, 0, 0, 4, 2]


In [6]:
for i in range(logn - 3):
    print(i)
    StepDown(i,f)
    StepDown(i,g)
print(f,g)

0
1
[7848524, 9771389, 12613644, 4650992, 9697475, 14086508, 7137276, 3869262, 6426026, 16291434, 4147197, 4650992, 4577081, 9490018, 7137276, 7919001, 10072857, 469411, 11827788, 12109847, 12183760, 7794719, 9623561, 7532072, 9174263, 6989456, 4933053, 12109847, 7063366, 2150421, 9623561, 14201331] [5120394, 3493877, 15173841, 11995716, 9791491, 1497107, 9751276, 5120397, 11640439, 3321816, 1587000, 2278832, 6969342, 15263730, 8852682, 11640442, 5120394, 11867305, 14387985, 15005905, 9529539, 16424276, 8170105, 5120397, 11640439, 14838668, 2372856, 4241213, 7231294, 336561, 6747607, 11640442]


In [7]:
h = [0]*n
for i in range(0,n,8):
    h[i:i+8] = TMVP_base(f[i:i+8],g[i:i+8],omegas[total_power[i>>3]])
print(h)

[4362643, 4750369, 2847582, 2972138, 427772, 639951, 9685096, 686609, 9182084, 12307976, 11253994, 7443362, 685702, 1673418, 8746004, 9556105, 4443219, 9175985, 9439968, 3034642, 13463677, 3045623, 3045801, 6684783, 15533128, 7286940, 9979630, 3310539, 2183434, 11401461, 12044505, 16593897]


In [8]:


StepUp(1, h)
StepUp(0, h)


In [9]:

Correct_poly = f_poly*g_poly
Correct_list = (list(Correct_poly))

print(Correct_list)
print(Correct_list == h)

[16760685, 16760734, 16760710, 16760795, 16760771, 16760738, 16760768, 16760765, 16760816, 16760802, 12, 27, 19, 16760832, 20, 49, 35, 101, 55, 43, 40, 53, 60, 97, 99, 115, 91, 67, 137, 81, 129, 111]
True


In [10]:
# Define parameters
p = 16760833
p0i = 4043292671  # -1/p mod 2^32
r2 = 13696128  # r^2 mod p

# Montgomery multiplication function
def montymul(a, b, p, p0i):
    z = a * b
    w = (z * p0i) & 0xFFFFFFFF  # w = (z * p0i) mod 2^32
    d = (z + w * p) >> 32
    if d >= p:
        d -= p
    return d

# Original omegas array
omegas = [
    1, 16366294, 3166450, 14192771, 8258568, 12296714, 7192835, 6511330, 
    9529539, 15271039, 12943322, 11758216, 5336149, 9342819, 2971251, 11783397, 
    9623559, 12437688, 16656576, 2368341, 13549951, 893592, 7228067, 843839, 
    9791491, 12287189, 10750218, 15574480, 16264742, 11000108, 11443476, 
    1529312, 16760832, 394539, 13594383, 2568062, 8502265, 4464119, 9567998, 
    10249503, 7231294, 1489794, 3817511, 5002617, 11424684, 7418014, 13789582, 
    4977436, 7137274, 4323145, 104257, 14392492, 3210882, 15867241, 9532766, 
    15916994, 6969342, 4473644, 6010615, 1186353, 496091, 5760725, 5317357, 
    15231521
]

# Compute the Montgomery form of omegas
omegas_mont = [montymul(omega, r2, p, p0i) for omega in omegas]

# Print the result
print("Montgomery form of omegas:")
print(omegas_mont)


Montgomery form of omegas:
[4194048, 14494886, 15152879, 3034156, 16101275, 9421437, 9406032, 3096948, 14719561, 3387958, 11631221, 11686617, 11930405, 1716427, 7304379, 12000572, 6994530, 5999281, 14509801, 13656077, 15369345, 11959950, 10601440, 1099823, 14899773, 2179276, 4598303, 16218269, 10059753, 365533, 9943678, 5884202, 12566785, 2265947, 1607954, 13726677, 659558, 7339396, 7354801, 13663885, 2041272, 13372875, 5129612, 5074216, 4830428, 15044406, 9456454, 4760261, 9766303, 10761552, 2251032, 3104756, 1391488, 4800883, 6159393, 15661010, 1861060, 14581557, 12162530, 542564, 6701080, 16395300, 6817155, 10876631]
